In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

from collections import Counter
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import numpy as np

In [0]:
train = pd.read_csv('/content/drive/My Drive/DLDatasets/AV-Innoplexus/train 2.csv')
test = pd.read_csv('/content/drive/My Drive/DLDatasets/AV-Innoplexus/test 2.csv')

In [4]:
train = train.drop(['Doc_ID'], axis = 1)
test = test.drop(['Doc_ID'], axis = 1)

train.head(5)

,id,Sent_ID,Word,tag
0,1,1,Obesity,O
1,2,1,in,O
2,3,1,Low-,O
3,4,1,and,O
4,5,1,Middle-Income,O


In [5]:
train.isnull().sum()

id           0
Sent_ID      0
Word       130
tag          0
dtype: int64

In [0]:
train = train.fillna('NoWord')
test = test.fillna('NoWord')

In [7]:
## get pos_tags
import nltk
nltk.download('averaged_perceptron_tagger')

def agg_func(x):
  for w, t in nltk.pos_tag([x]):
    return t

train['POS'] = train['Word'].apply(agg_func)
test['POS'] = test['Word'].apply(agg_func)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
train.head(5)

,id,Sent_ID,Word,tag,POS
0,1,1,Obesity,O,NN
1,2,1,in,O,IN
2,3,1,Low-,O,JJ
3,4,1,and,O,CC
4,5,1,Middle-Income,O,JJ


In [0]:
class GetSentences(object):
    
    def __init__(self, data, dataset_type):
        self.n_sent = 1
        self.data = data
        self.empty = False
        if dataset_type == 'train':
          agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), s['POS'].values.tolist(), s['tag'].values.tolist())]
        else:
          agg_func = lambda s: [(w, p) for w, p in zip(s['Word'].values.tolist(), s['POS'].values.tolist())]
        self.grouped = self.data.groupby('Sent_ID').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = GetSentences(train, 'train')
train_sentences = getter.sentences


getter = GetSentences(test, 'test')
test_sentences = getter.sentences

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        #print(word1)
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
X = [sent2features(s) for s in train_sentences]
y = [sent2labels(s) for s in train_sentences]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [18]:
y[:10]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],


In [12]:
!pip3 install sklearn_crfsuite

    100% |████████████████████████████████| 757kB 10.6MB/s 


In [0]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [19]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=30,
    all_possible_transitions=True
)
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=30,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [0]:
X_test = [sent2features(s) for s in test_sentences]

y_pred = crf.predict(X_test)

In [22]:
y_pred[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [0]:
flat_list = [item for sublist in y_pred for item in sublist]

In [24]:
flat_list[:10]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [25]:
test.head(5)

,id,Sent_ID,Word,POS
0,4543834,191283,CCCVA,NN
1,4543835,191283,",",","
2,4543836,191283,MANOVA,NN
3,4543837,191283,",",","
4,4543838,191283,my,PRP$


In [0]:
test['tag'] = pd.Series(flat_list)

In [27]:
test.head(5)

,id,Sent_ID,Word,POS,tag
0,4543834,191283,CCCVA,NN,O
1,4543835,191283,",",",",O
2,4543836,191283,MANOVA,NN,O
3,4543837,191283,",",",",O
4,4543838,191283,my,PRP$,O


In [28]:
result = test.drop(['Word', 'POS'], axis = 1)
result.head(5)

,id,Sent_ID,tag
0,4543834,191283,O
1,4543835,191283,O
2,4543836,191283,O
3,4543837,191283,O
4,4543838,191283,O


In [0]:
result.to_csv("/content/drive/My Drive/DLDatasets/AV-Innoplexus/result.csv", sep = ",", index=False, columns=["id", "Sent_ID", "tag"])